In [4]:
# Basic plots
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
from datacube.utils import masking
from datacube.utils.cog import write_cog
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map, rgb
from dea_tools.datahandling import mostcommon_crs

# EASI defaults
easinotebooksrepo = '/home/jovyan/easi-notebooks'
if easinotebooksrepo not in sys.path: sys.path.append(easinotebooksrepo)
from easi_tools import EasiDefaults, xarray_object_size, notebook_utils

In [5]:
# Data tools
import numpy as np
from datetime import datetime

# Datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Dask
from dask.distributed import Client, LocalCluster

In [6]:
easi = EasiDefaults()

family = 'sentinel-2'
product = easi.product(family)
display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

Successfully found configuration for deployment "asia"


Default sentinel-2 product for "asia": [s2_l2a](https://explorer.asia.easi-eo.solutions/products/s2_l2a)

In [7]:
# Local cluster
# Default is to run on a compute node with 28 GiB of available memory and 8 cores.
# We'll make that explicit here .. but this should be adjusted based on your workflow

cluster = LocalCluster(n_workers=2, threads_per_worker=4)
cluster.scale(n=2, memory="14GiB")
client = Client(cluster)
display(client)

dashboard_address = notebook_utils.localcluster_dashboard(client=client,server=easi.hub)
display(dashboard_address)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 124.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34375,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 124.00 GiB
Comm: tcp://127.0.0.1:41019,Total threads: 4
Dashboard: http://127.0.0.1:38053/status,Memory: 62.00 GiB
Nanny: tcp://127.0.0.1:42201,


'https://hub.asia.easi-eo.solutions/user/dhnghia/proxy/8787/status'

In [8]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

In [9]:
from utils import load_data_geo
import geopandas as gpd
from deafrica_tools.areaofinterest import define_area
from datacube.utils.geometry import Geometry
import xarray as xr
train_path = "train/Soc Trang_Traning.shp"
train = load_data_geo(train_path)
train.head()

,Name,geometry
0,Water1,POINT (620223.628 1072681.101)
1,Water1,POINT (621057.521 1071074.602)
2,Water1,POINT (621891.602 1069430.621)
3,Water1,POINT (622762.564 1068048.689)
4,Water1,POINT (623256.484 1066777.765)


In [10]:
train = train.to_crs('EPSG:4326')

In [11]:
train.head()

,Name,geometry
0,Water1,POINT (106.09593 9.70220)
1,Water1,POINT (106.10348 9.68765)
2,Water1,POINT (106.11104 9.67276)
3,Water1,POINT (106.11893 9.66024)
4,Water1,POINT (106.12340 9.64873)


In [12]:
train.head().explore(column="Name", legend=True)

In [13]:
min_date = '2022-01-01' # 2021-11-01
max_date = '2022-02-01' # 2022-01-01
product = 's2_l2a'

loaded_datasets = {}

# Iterate over each point in the GeoDataFrame
for idx, point in train.iterrows():
    # Create a bounding box around the point
    aoi = define_area(lat=point.geometry.y, lon=point.geometry.x, buffer=0)
    geopolygon = Geometry(aoi["features"][0]["geometry"], crs=train.crs)
    geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=train.crs)
    # Get the latitude and longitude range of the geopolygon
    lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
    lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])
    #print(geopolygon_gdf.total_bounds)
    query = {
            "product": product,
            "x": lon_range, 
            "y": lat_range,
            "time": (min_date, max_date),
    }
    ncrs = notebook_utils.mostcommon_crs(dc, query)
    # print(ncrs)
    query.update({
            "output_crs": ncrs,
            "resolution": (-10, 10),
            "dask_chunks": {'x': 2048, 'y': 2048}
     })
    # print(query)
    # print(qr)
    # break
    data = dc.load(**query)

    # Store the loaded dataset in the dictionary with a key based on the point index
    key = f'point_{idx + 1}'
    
    valid_mask = masking.valid_data_mask(data)
    
    measurement_info = dc.list_measurements().loc[query['product']]

    # Separate lists of measurement names and flag names
    measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
    flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index
    for flag in flag_names:
        notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    display(masking.describe_variable_flags(data[flag]))
    flags_def = masking.describe_variable_flags(data[flag]).values
    flags_def = flags_def.tolist()[0][1]
    flag_name = 'scl'
    flag_data = data[[flag_name]].where(valid_mask[flag_name]).persist()   # Dataset

    good_pixel_flags = [flags_def[str(i)] for i in [4, 5, 6]]

    good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)
    rs = []
    for layer_name in ['red', 'green', 'blue', 'nir']:

        # Get scaling and offset values from product description
        scale = measurement_info.loc[layer_name].scale_factor
        offset = measurement_info.loc[layer_name].add_offset

        # Apply valid mask and good pixel mask
        layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scale + offset
        layer = layer.persist()
        rs.append(layer)
    result = rs[0].merge(rs[1])
    result = result.merge(rs[2])
    result = result.merge(rs[3])
    
    loaded_datasets[key] = result

,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


/env/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


...

,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


In [14]:
len(loaded_datasets)

1051

In [15]:
from deafrica_tools.bandindices import calculate_indices
import numpy as np

In [16]:
ndivi_dataset = {}
for i in loaded_datasets.keys():
    tmp = calculate_indices(loaded_datasets[i], index='NDVI', satellite_mission='s2')
    ndivi_dataset[i] = tmp.NDVI.mean(dim='time')

In [17]:
ndivi_dataset["point_1"].values

array([[0.8544633]], dtype=float32)

In [18]:
# ndivi_dataset['point_1'].plot(cmap='RdYlGn',
#            size=6, vmin=-2, vmax=2,
# col_wrap=2)

In [19]:
ndivi_dataset['point_1'].values

array([[0.8544633]], dtype=float32)

In [20]:
labels = train.Name.values

In [21]:
X = []
for i in ndivi_dataset.keys():
    X.append(ndivi_dataset[i].values)

2023-12-22 00:48:48,031 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [22]:
max(arr.shape for arr in X)

(1, 1)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [24]:
label_encoder = LabelEncoder()

# Fit and transform the labels
numeric_labels = label_encoder.fit_transform(labels)

In [25]:
label_mapping = dict(zip(labels, numeric_labels))

In [26]:
X_flat = np.vstack([arr.flatten() for arr in X])

In [27]:
vt_nan = [i[0] for i in np.argwhere(np.isnan(X_flat))]

In [28]:
x_new = []
lb_new = []
for i in range(len(X)):
    if i not in vt_nan:
        x_new.append(X[i])
        lb_new.append(numeric_labels[i])

In [29]:
x_new = np.vstack([arr.flatten() for arr in x_new])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(x_new, lb_new, test_size=0.3, random_state=42)

In [31]:
model = RandomForestClassifier(n_estimators=150, random_state=42, criterion='gini', max_depth=2)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, n_estimators=150, random_state=42)

In [32]:
predictions = model.predict(X_test)

In [33]:
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 0.32608695652173914


In [ ]:
predictions

In [34]:
# Vietnam
min_longitude, max_longitude = (105.5, 106.4)
min_latitude, max_latitude = (9.2, 10.0)
min_date = '2022-01-01' # 2021-11-01
max_date = '2022-02-01' # 2022-01-01
product = 's2_l2a'

query1 = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
}

# Most common CRS
native_crs = notebook_utils.mostcommon_crs(dc, query1)

query1.update({
    'output_crs': native_crs,               # EPSG code
    'resolution': (-10, 10),                # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'x': 2048, 'y': 2048},  # Dask chunks
})


In [35]:
# Load data
data = dc.load(**query1)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)
notebook_utils.heading('Valid data masks for each variable')
display(valid_mask)

<xarray.Dataset>
Dimensions:      (time: 12, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-01-02T03:25:18.223000 ... 2022-02...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables: (12/15)
    coastal      (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    rededge1     (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    rededge2     (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    ...           ...
    nir09        (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir16       (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir22       (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    aot          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    wvp          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

<xarray.Dataset>
Dimensions:      (time: 12, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-01-02T03:25:18.223000 ... 2022-02...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables: (12/15)
    coastal      (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    rededge1     (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    rededge2     (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    ...           ...
    nir09        (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir16       (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir22       (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    aot          (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    wvp          (time, y, x) bool dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [36]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[query1['product']]
notebook_utils.heading(f'Measurement table for product: {query1["product"]}')
display(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
display(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    display(masking.describe_variable_flags(data[flag]))

,name,dtype,units,nodata,flags_definition,aliases,add_offset,scale_factor
measurement,,,,,,,,
coastal,coastal,uint16,1,0,NaN,"[band_01, B01, coastal_aerosol]",-0.1,0.0001
blue,blue,uint16,1,0,NaN,"[band_02, B02]",-0.1,0.0001
green,green,uint16,1,0,NaN,"[band_03, B03]",-0.1,0.0001
red,red,uint16,1,0,NaN,"[band_04, B04]",-0.1,0.0001
rededge1,rededge1,uint16,1,0,NaN,"[band_05, B05, red_edge_1]",-0.1,0.0001
rededge2,rededge2,uint16,1,0,NaN,"[band_06, B06, red_edge_2]",-0.1,0.0001
rededge3,rededge3,uint16,1,0,NaN,"[band_07, B07, red_edge_3]",-0.1,0.0001
nir,nir,uint16,1,0,NaN,"[band_08, B08, nir_1]",-0.1,0.0001
nir08,nir08,uint16,1,0,NaN,"[band_8a, B8A, nir_2]",-0.1,0.0001


,group,names
0,Measurement names,"coastal, blue, green, red, rededge1, rededge2,..."
1,Flag names,scl


,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


In [37]:
flags_def = masking.describe_variable_flags(data[flag]).values
flags_def = flags_def.tolist()[0][1]
# Make SCL flags image
flag_name = 'scl'
flag_data = data[[flag_name]].where(valid_mask[flag_name]).persist()   # Dataset
display(flag_data)
# Create Mask layer

good_pixel_flags = [flags_def[str(i)] for i in [4, 5, 6]]

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> DataArray
# display(good_pixel_mask)  # Type: bool

<xarray.Dataset>
Dimensions:      (time: 12, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-01-02T03:25:18.223000 ... 2022-02...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    scl          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [38]:
# Select a layer and apply masking and scaling, then persist in dask
# layer_name = 'red'
rs = []
for layer_name in ['red', 'green', 'blue', 'nir']:

    # Get scaling and offset values from product description
    scale = measurement_info.loc[layer_name].scale_factor
    offset = measurement_info.loc[layer_name].add_offset

    # Apply valid mask and good pixel mask
    layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scale + offset
    layer = layer.persist()
    rs.append(layer)

In [39]:
import xarray as xr
result = rs[0].merge(rs[1])
result = result.merge(rs[2])
result = result.merge(rs[3])

In [40]:
ds1 = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = ds1["NDVI"]
average_ndvi = ndvi.mean(dim='time')

In [41]:
average_ndvi

<xarray.DataArray 'NDVI' (y: 8874, x: 9902)>
dask.array<mean_agg-aggregate, shape=(8874, 9902), dtype=float32, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648

In [43]:
data_array = xr.DataArray(np.zeros((8874, 9902)), dims=('y', 'x'))

In [ ]:
for i in range(average_ndvi.values.shape[0]):
    for j in range(average_ndvi.values.shape[1]):
        x = average_ndvi.values[i][j]
        if np.isnan(x):
            data_array[i][j] = -1
        else:
            data_array[i][j] = model.predict([[x]])[0]

In [ ]:
average_ndvi["labels"] = data_array

In [ ]:
data_array